In [1]:
from collections import defaultdict
from dwave.system import DWaveSampler, EmbeddingComposite
import numpy as np
import random
import math
import multiprocessing

In [2]:
# Randomizes input for Frobenius problem
def nums(n=5,r=100):
    out = list()
    l = list(range(2,r))
    gcd = l.pop()
    out.append(gcd)
    while True:
        nl = random.sample(l,n)
        if math.gcd(*nl)==1:
            return nl
# Edge to qbit number encoding
def f(i: int ,j: int) -> int:
    global n
    return n*i+j
# Takes edge's index in B and the vertex at which it must end is 'i'
# Returns the vertex from which the corresponding edge goes to i
def jtoi(j: int ,i: int) -> int:
    global B
    global a0
    return (a0-B[j][0]+i)%a0

In [3]:
def ConstraintGen(B: list,n: int,Y: range,rn: range,c: list,a0: int,i: int,Q: defaultdict) -> None:
    # B is list of (j,w)
    # a0 is the smallest of input
    # i is the vertex in consideration
    # Q is the input to DWAVE
    # flag : 1 means i is source
    #        2 means i is destination
    #        3 means i is other vertex
    print('in')
    a = 0
    b = 0
    flag=int(i!=0)+1
    if i==0:
        a=-c[1]
        b=3*c[1]
        for i in rn:
            for j in Y:
                tmp = f(i,j)
                Q[(tmp,tmp)]=B[j][1]*c[0]
    else:
        a=c[3]
        b=c[3]
    
    for j in Y:
        z1=f(i,j)
        q1=jtoi(j,i)
        z2=f(q1,j)
        Q[(z1,z1)]+=a
        Q[(z2,z2)]+=b
        for k in range(j+1,n):
            q2 = jtoi(k,i)
            # i->k
            z3=f(i,k)
            # k->i
            z4=f(q2,k)
            # i->j i->k
            Q[(z1,z3)]+=2*c[flag]
            # j->i k->i
            Q[(z2,z4)]+=2*c[flag]
        for k in Y:
            q2=jtoi(k,i)
            # k->i
            z4=f(q2,k)
            # i->j k->i
            Q[(z1,z4)]+=-2*c[flag]

In [4]:
def QShortestPath(Y: range,rn: range,d: int,Q: list,num_reads: int):
    Q1=defaultdict(int)
    for i in rn:
        for a,b in Q[i].items():
            Q1[a]+=b
    for j in Y:
        z1=f(i,j)
        q1=jtoi(j,i)
        z2=f(q1,j)
        Q1[(z1,z1)]+=3*c[2]
        Q1[(z2,z2)]+=-c[2]
    s = EmbeddingComposite(DWaveSampler())
    ss = s.sample_qubo(Q1,num_reads=num_reads)
    O=ss.first[0]
    f1=0
    for i in range(len(B)*a0):
        if O[i]==1:
            f1+=B[i%len(B)][1]
    return f1

In [5]:
def QCirculantDiameter(B,n,Y,rn,c,a0,i):
    pathln=list()
    e = concurrent.futures.ThreadPoolExecutor()
    for i in rn:
        q=defaultdict(int)
        t=e.submit(ConstraintGen,B,n,Y,rn,c,a0,i,q)
        T.append(t)
        Q.append(q)
    for i in rn:
        T[i].result()
    T.clear()
    for i in range(1,a0):
        t=e.submit(QShortestPath,Y,rn,i,Q,999)
        T.append(t)
        Q.append(q)
    for i in range(a0-1):
        x=T[i].result()
        pathln.append(x)
        print(x)
    T.clear()
    return max(pathln)

In [6]:
#A=[13,17,19,23,31,37,41,43]
#A=nums()
A=[5,7,11,17]
print('A = ',A)
A.sort(reverse=True)
a0=A.pop()
Q=list()
T=list()
B=dict()
for x in A:
    i = x%a0
    if i != 0:
        B[i]=x
B=list(B.items())
print(B)
n=len(B)
Y=range(n)
rn=range(a0)
c1=a0*sum([w for j,w in B])
c=[1,c1,c1,c1]
print('a0 = ',a0)
m = QCirculantDiameter(B,n,Y,rn,c,a0,i)
print('f(A) = ',m-a0)

A =  [5, 7, 11, 17]
[(2, 7), (1, 11)]
in
in
in
in
in
0
0
0
0
a0 =  5
f(A) =  -5
